# 邮件内容提取摘要
## Version 2.0
### 读取CSV文件，对Content生成摘要，以修改为AzureOpenAI调用

In [1]:
import os
import pandas as pd
from langchain.llms import AzureOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain 

In [2]:
base_dir='./email'
questionsfile='newdata.csv'
f = open(os.path.join(base_dir, questionsfile), encoding="utf-8")
questionsdf=pd.read_csv(f)
# 第0列为序号无意义，删除第0列unnamed
questionsdf.drop(questionsdf.columns[0],axis=1,inplace=True)
questionsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  580 non-null    object
 1   Body     576 non-null    object
 2   Content  576 non-null    object
dtypes: object(3)
memory usage: 13.7+ KB


## 最简单的方式生成摘要
### 可更换chain_type的参数，测试那种类型摘要结果好

In [13]:
# 初始化大模型，换成你自己的大模型调用
openai_api_key = "5d4d1d30e5704cc3b2812f8304e432fe" 

llm = AzureOpenAI(
    openai_api_key=openai_api_key, 
    deployment_name="testchat",
    model_name="gpt-35-turbo",
    openai_api_version="2023-07-01-preview",
    temperature=0.7, 
#    streaming=True, 
#    callbacks=[StreamingStdOutCallbackHandler()]
    )
# 初始化拆分器
text_splitter = CharacterTextSplitter()

def get_summary(content):
    # 拆分文档
    texts = text_splitter.split_text(content)
    # 将拆分后的文本转成文档
    docs = [Document(page_content=t) for t in texts[:3]]
    # 生成摘要
    #load_summarize_chain的chain_type可以取值stuff、map_reduce、refine
    chain = load_summarize_chain(llm, chain_type="stuff")
    summary = chain.run(docs)
    
    return summary

questionsdf['summary'] = questionsdf['Content'].apply(get_summary)
    # 拆分文本
questionsdf.to_csv(os.path.join(base_dir,"newdata.csv"))
questionsdf['summary']



0      test
1      test
2      test
3      test
4      test
       ... 
575    test
576    test
577    test
578    test
579    test
Name: summary, Length: 580, dtype: object

## 通过利用自定义提示实现chatgpt翻译摘要
### 可更换chain_type的参数，测试那种类型摘要结果好

In [ ]:
# 初始化大模型，换成你自己的大模型调用
openai_api_key = "5d4d1d30e5704cc3b2812f8304e432fe" 

llm = AzureOpenAI(
    openai_api_key=openai_api_key, 
    deployment_name="testchat",
    model_name="gpt-35-turbo",
    openai_api_version="2023-07-01-preview",
    temperature=0.7, 
#    streaming=True, 
#    callbacks=[StreamingStdOutCallbackHandler()]
    )
# 初始化拆分器
text_splitter = CharacterTextSplitter()

def get_summary(content):
    # 拆分文档
    texts = text_splitter.split_text(content)
    # 将拆分后的文本转成文档
    docs = [Document(page_content=t) for t in texts[:3]]
    # 生成摘要
    #load_summarize_chain的chain_type可以取值stuff、map_reduce、refine
    prompt_template = """Write a concise summary of the following:

{text}

CONCISE SUMMARY IN ENGLISH:"""
    # 上面的prompt是要用英语语做摘要
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
    # summarize_chain
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
    summary = chain.run(docs)
 
    return summary

questionsdf['summary'] = questionsdf['Content'].apply(get_summary)
    # 拆分文本
questionsdf.to_csv(os.path.join(base_dir,"newdata.csv"))
questionsdf['summary']